In [1]:
!python util/common.py

In [2]:
!python util/ids.py

In [3]:
!python util/corrections.py

Loading files in: data/jec
Loading file: data/jec/Summer16_07Aug2017_V11_MC_L2Relative_AK4PFPuppi.jec.txt
Loading file: data/jec/Autumn18_V19_MC_L2L3Residual_AK4PFPuppi.jec.txt
Loading file: data/jec/Summer16_07Aug2017_V11_MC_L1FastJet_AK4PFPuppi.jec.txt
Loading file: data/jec/Summer16_07Aug2017_V11_MC_L3Absolute_AK4PFPuppi.jec.txt
Loading file: data/jec/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt
Loading file: data/jec/Summer16_07Aug2017_V11_MC_L2L3Residual_AK4PFPuppi.jec.txt
Loading file: data/jec/Autumn18_V19_MC_L2Residual_AK4PFPuppi.jec.txt
Loading file: data/jec/Fall17_17Nov2017_V32_MC_L1FastJet_AK4PFPuppi.jec.txt
Loading file: data/jec/Autumn18_V19_MC_L1FastJet_AK4PFPuppi.jec.txt
Loading file: data/jec/Autumn18_V19_MC_L2Relative_AK4PFPuppi.jec.txt
Loading file: data/jec/Fall17_17Nov2017_V32_MC_L2L3Residual_AK4PFPuppi.jec.txt
Loading file: data/jec/Fall17_17Nov2017_V32_MC_L3Absolute_AK4PFPuppi.jec.txt
Loading file: data/jec/Summer16_07Aug2017_V11_MC_L2Residual_AK4PFPuppi

In [6]:
!python processors/darkhiggs.py --year 2018

In [7]:
import pyspark.sql
import os
import sys
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize

# The following line is necessary because we're working in a
# virtualenv. Without it, executors will use the wrong interpreter!
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

#    .config("spark.driver.extraClassPath","/home/cms.lgray/sparkMeasure/target/scala-2.12/spark-measure_2.12-0.14-SNAPSHOT.jar") \
#    .master('local[1]') \
#    .master('spark://cmsspark-submit.fnal.gov:7077') \
#    .config('spark.executorEnv.X509_PROXY',"%s" % os.environ['X509_USER_PROXY']) \
#    .config('spark.files','%s' % os.environ['X509_USER_PROXY']) \

conf = pyspark.sql.SparkSession.builder \
    .appName('decaf-%s' % guid()) \
    .config('spark.driver.cores',8) \
    .config('spark.driver.memory','10g') \
    .config('spark.executor.memory', "20g") \
    .config('spark.debug.maxToStringFields',1000) \
    .config('spark.executorEnv.XRD_RUNFORKHANDLER','1') \
    .config('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT','1') \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config("spark.blockManager.port", "8000") \
    .config('spark.shuffle.service.enabled','true') \
    .config('spark.scheduler.mode', 'FAIR') \
    .config('spark.executor.instances','1') \
    .config('spark.dynamicAllocation.enabled','true') \
    .config('spark.dynamicAllocation.executorIdleTimeout','1m') \
    .config('spark.dynamicAllocation.cachedExecutorIdleTimeout','10m') \
    .config('spark.dynamicAllocation.minExecutors','1') \
    .config('spark.dynamicAllocation.maxExecutors','80') \
    .config('spark.sql.execution.arrow.pyspark.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.hadoop.fs.xrootd.read.buffer', 100 * 1048576) \
    .config('spark.hadoop.fs.xrootd.write.buffer', 100 * 1048576) \
    .config('spark.driver.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar')
session = _spark_initialize(config=conf,log_level='WARN',laurelin_version='1.1.1-SNAPSHOT')
sc = session.sparkContext
sc.setLogLevel("WARN")
spark = session

In [13]:
import pyspark.sql.functions as fn
from tqdm import tqdm
import json

fileslice = slice(None)
with open("metadata/2018.json") as fin:
    samplefiles = json.load(fin)

filelist = {}
i = 0
for dataset, info in samplefiles.items():
    files = []
    for file in info['files'][fileslice]:
        files.append(file)
        filelist[dataset] = {'files': files, 'treename': 'Events'}
        
        #if 'MET____0_' in dataset: 
            #filelist[dataset] = {'files': files, 'treename': 'Events'}
print(filelist['MET____0_'])

{'files': ['/home01/x1797a02/nurion/Samples/nano_1.root', '/home01/x1797a02/nurion/Samples/nano_10.root', '/home01/x1797a02/nurion/Samples/nano_100.root', '/home01/x1797a02/nurion/Samples/nano_101.root', '/home01/x1797a02/nurion/Samples/nano_102.root', '/home01/x1797a02/nurion/Samples/nano_103.root', '/home01/x1797a02/nurion/Samples/nano_104.root', '/home01/x1797a02/nurion/Samples/nano_105.root', '/home01/x1797a02/nurion/Samples/nano_106.root', '/home01/x1797a02/nurion/Samples/nano_107.root'], 'treename': 'Events'}


In [9]:
from coffea.util import load
processor_instance = load('data/darkhiggs2018.processor')

In [10]:
partitionsize = 500
thread_workers = 8
#import os
#os.environ["ARROW_PRE_0_15_IPC_FORMAT"]="1"

In [11]:
import time
from coffea.processor import run_spark_job
from coffea.processor.spark.spark_executor import spark_executor
import pyspark.sql.functions as F

def presel_filter(df):
    df = df.filter(F.expr('exists(AK15Puppi_pt, x -> x > 160)'))
    return df

tic = time.time()
final_accumulator = run_spark_job(filelist, processor_instance, spark_executor, 
                                  spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                  executor_args={'file_type': 'root', 'cache': False, 'df_xform':presel_filter})
dt = time.time() - tic

/opt/conda/lib/python3.6/site-packages/coffea/processor/executor.py:941: UserWarning: If you are using pyarrow >= 0.15.0, make sure to set ARROW_PRE_0_15_IPC_FORMAT=1 in your environment!
  warnings.warn('If you are using pyarrow >= 0.15.0, make sure to set %s=%s in your environment!' % arrow_env)
Processing:   0%|          | 0/1 [00:00<?, ?datasets/s]/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:2146: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o746.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:874)
	at org.apache.spark.api.p

Processing:   0%|          | 0/1 [02:09<?, ?datasets/s]


Py4JJavaError: An error occurred while calling o746.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:874)
	at org.apache.spark.api.python.PythonServer.getResult(PythonRDD.scala:870)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConne

In [8]:
!/usr/bin/python -V

Python 2.7.17
